# Starting Lyrics Processing

###### Importing all libraries needed

In [53]:
import numpy as np

import nltk

from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import copy
from scipy.stats import friedmanchisquare as friedman
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import ttest_ind as ttest
from scipy.stats import ttest_ind_from_stats as ttest2
from scipy.stats import f_oneway as f_oneway
from matplotlib import pyplot as plt
import csv
import os
from bs4 import BeautifulSoup
import codecs
from tqdm import tqdm_notebook, tnrange
import math

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipetanios/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

###### getting all the lyrics

In [33]:
tracks_names = []
tracks_lyrics = []

for root, dirs, files in os.walk("."):
    for f in files:
        if f[-3:] == "txt":
            tracks_names.append(f)
            tracks_path = os.path.join(root, f)
            track_single_lyric =  open (tracks_path, 'r')
            tracks_lyrics.append(track_single_lyric.read().lower())


for i in range(len(tracks_names)):
    tracks_names[i] = tracks_names[i][:-4]

print (tracks_lyrics)
print (tracks_names)
print (len(tracks_names))


[' madalena o meu peito percebeu que o mar é uma gota comparado ao pranto meu.  fique certa quando o nosso amor desperta logo o sol se desespera e se esconde lá na serra.  madalena o que é meu não se divide nem tão pouco se admite quem do nosso amor duvide.  até a lua se arrisca num palpite que o nosso amor existe forte ou fraco alegre ou triste.  ', ' depois de trabalhar toda a semana meu sábado não vou desperdiçar já fiz o meu programa pra esta noite e sei por onde começar  um bom lugar para encontrar: copacabana prá passear à beira-mar: copacabana depois num bar à meia-luz: copacabana eu esperei por essa noite uma semana  um bom jantar depois de dançar: copacabana um só lugar para se amar: copacabana a noite passa tão depressa, mas vou voltar se pra semana eu encontrar um novo amor: copacabana  ', ' pai, afasta de mim esse cálice pai, afasta de mim esse cálice pai, afasta de mim esse cálice de vinho tinto de sangue  pai, afasta de mim esse cálice pai, afasta de mim esse cálice pai, 

<h6>creating dictionaries associating lyrics names and their percieved emotion</h6>

In [34]:
dataset = {}
with open ('base.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        dataset[row[0][:-4]] = row[2]
    
print (dataset)

{'Onde_anda_voce_Toquinho_Vinicius_de_moraes': 'Sad', 'eu_nao_existo_sem_voce_-_Maria_Bethania': 'Sad', 'Sabado_em_Copacabana': 'Happy', 'Tarde_em_Itapoa_-_Vinicius_e_Toquinho': 'Happy', 'Elis_Regina_-_Carinhoso_(Meu_Coracao)': 'Sad', 'aguas_de_Marco': 'Happy', 'A_Felicidade': 'Sad', 'Joao_Gilberto_-_Ela_e_Carioca': 'Happy', 'Garota_de_Ipanema': 'Happy', 'Marcos_Valle_-_Samba_de_verao': 'Happy', 'Carlos_Lyra_-_Influencia_do_Jazz': 'Happy', 'Calice': 'Sad', 'Elis_Regina_-_Madalena': 'Happy', 'Sergio_Mendes_+_Brasil_66_-_Mas_Que_Nada': 'Happy', 'Joao_Gilberto_-_Lobo_Bobo': 'Happy', 'Elis_Regina_-_Tristeza': 'Sad', 'Jorge_Ben_Jor_-_Que_Maravilha': 'Happy', 'Joao_Gilberto_-_O_Barquinho': 'Happy', 'Gal_Costa_-_Anos_Dourados': 'Sad', 'Gal_Costa_-_Se_Todos_Fossem_Iguais_A_Voce': 'Sad', 'Gilberto_Gil_-_Rosa_Morena': 'Happy', 'Toquinho_-_Meditacao': 'Sad', 'Samba_Em_Preludio_-_Vinicius_de_Moraes_,Maria_Creuza_y_Toquinho': 'Sad', 'Samba_de_Uma_Nota_So': 'Happy', 'Elis_Regina_-_O_Bebado_e_a_Equil

<h6>Removing Stopwords</h6>

In [35]:
#getting portuguese stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')
print (stopwords)

#deciding my own stopwords, because the ones describes in nltk includes some verbs that i would like to not use

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos

In [36]:
stopwords = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 
             'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 
             'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu',
             'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também',
             'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 
             'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 
             'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 
             'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 
             'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 
             'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 
             'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 
             'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo']

#making stopwords findable
for i in range(len(stopwords)):
    stopwords[i] = ' '+stopwords[i] + ' '

print (stopwords)

#removing stopwords

for lyric in range(len(tracks_lyrics)):
    for stopword in stopwords:        
        tracks_lyrics[lyric] = tracks_lyrics[lyric].replace(stopword, ' ')
        
print(tracks_lyrics)

[' de ', ' a ', ' o ', ' que ', ' e ', ' do ', ' da ', ' em ', ' um ', ' para ', ' com ', ' não ', ' uma ', ' os ', ' no ', ' se ', ' na ', ' por ', ' mais ', ' as ', ' dos ', ' como ', ' mas ', ' ao ', ' ele ', ' das ', ' à ', ' seu ', ' sua ', ' ou ', ' quando ', ' muito ', ' nos ', ' já ', ' eu ', ' também ', ' só ', ' pelo ', ' pela ', ' até ', ' isso ', ' ela ', ' entre ', ' depois ', ' sem ', ' mesmo ', ' aos ', ' seus ', ' quem ', ' nas ', ' me ', ' esse ', ' eles ', ' você ', ' essa ', ' num ', ' nem ', ' suas ', ' meu ', ' às ', ' minha ', ' numa ', ' pelos ', ' elas ', ' qual ', ' nós ', ' lhe ', ' deles ', ' essas ', ' esses ', ' pelas ', ' este ', ' dele ', ' tu ', ' te ', ' vocês ', ' vos ', ' lhes ', ' meus ', ' minhas ', ' teu ', ' tua ', ' teus ', ' tuas ', ' nosso ', ' nossa ', ' nossos ', ' nossas ', ' dela ', ' delas ', ' esta ', ' estes ', ' estas ', ' aquele ', ' aquela ', ' aqueles ', ' aquelas ', ' isto ', ' aquilo ']
[' madalena peito percebeu mar é gota compara

<h6>Creating dictionary with all the words</h6>

In [47]:
word_dic = {}
word_count = 0

for i in range(len(tracks_lyrics)):
    for word in tracks_lyrics[i].split():
        word_count += 1
        if word not in word_dic:
            word_dic[word] = 1
        else:
            word_dic[word] = 1 + word_dic[word]
            
print (word_count)
print (word_dic)
most_common = 0
index = 0
for i in word_dic:
    if word_dic[i] > most_common:
        most_common = word_dic[i]
        index = i
# print(most_common)
# print (index)


3302
{'falar': 16, 'rio': 3, 'voz': 3, 'encontrou': 1, 'gravador': 2, 'profundo,': 1, 'somos': 6, 'caminho': 12, 'brilhando,': 1, 'febre': 1, 'boemia': 1, 'própria': 1, 'misto': 2, 'ar': 3, 'eh,': 1, 'cidade': 11, 'morena!': 2, 'pétala': 1, 'veneno': 1, 'cai,': 1, 'canhões': 2, 'sei': 20, 'traz': 5, 'nada.': 2, 'envolver': 2, 'vendo': 1, 'apareceu': 1, 'saúde': 1, 'matinês': 1, 'pileque': 1, 'meia-luz:': 1, 'molhados,': 1, 'modernizando,': 1, 'coleira': 1, 'talvez': 2, 'praia': 1, 'ouvindo': 8, 'inevitável': 1, 'loucas,': 1, 'dezembro': 1, 'ainda': 8, 'cowboy': 1, 'luto': 1, 'tantos': 1, 'ficava': 1, 'predestinada': 1, 'desespera': 1, 'próximo,': 1, 'procurou': 1, 'prá': 6, 'costão': 1, 'mãe': 1, 'triste?': 1, 'esquecendo': 2, 'deixou': 2, 'balanço': 1, 'chão,': 1, 'sol': 15, 'pousar...': 1, 'torto': 3, 'ir,': 2, 'belo': 2, 'morreu': 1, 'rolha': 1, 'parar': 3, 'brasil': 2, 'encontrar': 2, 'abrir': 1, 'jovens': 1, 'engraçado': 1, 'mato': 2, 'jardim': 1, 'atiradeira': 1, 'surgiu,': 1, 'n

<h6>Counting TF for each word in each lyric</h6>

In [46]:
tracks_tf = []
for i in range(len(tracks_lyrics)):
    dic = word_dic.copy()
    for j in dic:
        dic[j] = 0
    #counting all the words in a lyric
    word_count = 0
    for word in tracks_lyrics[i].split():
        word_count += 1
    #getting how many of the same word is in the dictionary
    for word in tracks_lyrics[i].split():
        dic[word] = 1 + dic[word]
    #calculating TF
    for key in dic:
        dic[key] = dic[key]/word_count
    #printing the pairs of key and value
#     soma = 0
#     for k, v in dic.items():
#     #    print(k, v)
#         soma += v
#         print(soma)    
    tracks_tf.append(dic)
    
print(tracks_tf)

[{'fechado': 0.0, 'voz': 0.0, 'caminho': 0.0, 'brilhando,': 0.0, 'boemia': 0.0, 'dia,': 0.0, 'ar': 0.0, 'morena!': 0.0, 'pena': 0.0, 'canhões': 0.0, 'paralisada': 0.0, 'envolver': 0.0, 'matinês': 0.0, 'pileque': 0.0, 'molhados,': 0.0, 'modernizando,': 0.0, 'talvez': 0.0, 'ouvindo': 0.0, 'ri': 0.0, 'ficava': 0.0, 'desespera': 0.027777777777777776, 'canta': 0.0, 'preguiça': 0.0, 'esquecendo': 0.0, 'chamar': 0.0, 'sol': 0.027777777777777776, 'terminar': 0.0, 'torto': 0.0, 'belo': 0.0, 'morreu': 0.0, 'rolha': 0.0, 'pingando,': 0.0, 'brasil': 0.0, 'encontrar': 0.0, 'apanhar!': 0.0, 'jovens': 0.0, 'mato': 0.0, 'tem': 0.0, 'nunca': 0.0, 'esquecer': 0.0, 'edifícios': 0.0, 'atordoado': 0.0, 'quiser': 0.0, 'cai': 0.0, 'pele': 0.0, 'luz,': 0.0, 'tinto': 0.0, 'querer': 0.0, 'salvar': 0.0, 'cara': 0.0, 'primeira': 0.0, 'pluma': 0.0, 'percebeu': 0.027777777777777776, 'cheiro': 0.0, 'cuidado,': 0.0, 'suspiro': 0.0, 'vai.': 0.0, 'retirantes': 0.0, 'óleo': 0.0, 'regato,': 0.0, 'encaminham': 0.0, 'quase

<h6>Calculating IDF for each word</h6>

In [58]:
#number of documents :
documents = len(tracks_tf)

document_frequency = {}
#to get idf we must calculate log(#documents/#documents_with_term)
#so we should go through the whole dictionary and get the ammount of documents
for i in word_dic:
    count = 0
    for k in range(len(tracks_lyrics)):
        if i in tracks_lyrics[k]:
            count+=1
    #count = number of doc
    document_frequency[i] = math.log10(documents/count)
    
print(document_frequency)

{'falar': 0.7231036840527382, 'rio': 0.8692317197309762, 'voz': 1.2671717284030137, 'encontrou': 1.568201724066995, 'gravador': 1.568201724066995, 'profundo,': 1.568201724066995, 'somos': 1.568201724066995, 'caminho': 0.7231036840527382, 'brilhando,': 1.568201724066995, 'boemia': 1.568201724066995, 'dia,': 1.568201724066995, 'ar': 0.03672280702473983, 'eh,': 1.568201724066995, 'cidade': 0.7900504736833514, 'morena!': 1.568201724066995, 'pétala': 1.568201724066995, 'veneno': 1.568201724066995, 'estrada': 1.2671717284030137, 'canhões': 1.2671717284030137, 'sei': 0.422073688388757, 'traz': 0.8692317197309762, 'ficou': 1.2671717284030137, 'envolver': 1.568201724066995, 'vendo': 1.0910804693473326, 'apareceu': 1.568201724066995, 'saber': 1.568201724066995, 'admirar': 1.568201724066995, 'matinês': 1.568201724066995, 'pileque': 1.568201724066995, 'meia-luz:': 1.568201724066995, 'molhados,': 1.568201724066995, 'modernizando,': 1.568201724066995, 'encontrar:': 1.568201724066995, 'talvez': 1.267

<h6>Calculating TFIDF for each word in each lyric</h6>

In [62]:
tracks_tfidf = []
for i in range(len(tracks_tf)):
    aux={}
    for word in tracks_tf[i]:
        aux[word] = tracks_tf[i][word]*document_frequency[word]
        #aprint(aux[word])
    tracks_tfidf.append(aux)
    
print(tracks_tfidf)

[{'fechado': 0.0, 'caminho': 0.0, 'voz': 0.0, 'encontrou': 0.0, 'pena': 0.0, 'brota': 0.0, 'somos': 0.0, 'rio': 0.0, 'sumiu': 0.0, 'brilhando,': 0.0, 'boemia': 0.0, 'dia,': 0.0, 'ar': 0.0, 'eh,': 0.0, 'cidade': 0.0, 'morena!': 0.0, 'cuidado,': 0.0, 'veneno': 0.0, 'gravador': 0.0, 'canhões': 0.0, 'sei': 0.0, 'traz': 0.0, 'nada.': 0.0, 'envolver': 0.0, 'vendo': 0.0, 'apareceu': 0.0, 'saber': 0.0, 'dá': 0.0, 'foi': 0.0, 'matinês': 0.0, 'pileque': 0.0, 'meia-luz:': 0.0, 'molhados,': 0.0, 'arriscou': 0.0, 'coleira': 0.0, 'talvez': 0.0, 'praia': 0.0, 'pingo': 0.0, 'ri': 0.0, 'loucas,': 0.0, 'dezembros': 0.0, 'laço,': 0.0, 'inevitável': 0.0, 'cowboy': 0.0, 'luto': 0.0, 'ficava': 0.0, 'predestinada': 0.0, 'desespera': 0.04356115900186097, 'próximo,': 0.0, 'começar': 0.0, 'canta': 0.0, 'costão': 0.0, 'mãe': 0.0, 'triste?': 0.0, 'esquecendo': 0.0, 'chamar': 0.0, 'chão,': 0.0, 'sol': 0.010891957361425382, 'terminar': 0.0, 'torto': 0.0, 'belo': 0.0, 'morreu': 0.0, 'sim': 0.0, 'vive': 0.0, 'pingand